In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.spatial.distance import squareform, pdist

In [3]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [4]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [5]:
import sys

In [6]:
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
from sklearn.metrics import f1_score

In [8]:
def labelClusters(clusters, target):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myClasses = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myClasses.append(target[j]) 
        resultList.append(max(set(myClasses), key=myClasses.count))
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []
        
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [9]:
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [10]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [11]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    menor = len(clusters)
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            menor = i
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters[j]==menor):
            myList.append(j)
            
    return myList

In [19]:
def genieLinkage(data, limiar, clusterNumber):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(counting(clusters[k-1])) <= limiar):
            for i in range(0, data.shape[0]):
                for j in range(i, data.shape[1]):
                    if(data[i][j]<min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (len(set(clusters[k])) == clusterNumber):
            return clusters[k]

In [43]:
l = [[0, 1, 'f'], [4, 2, 't'], [9, 4, 'afsd']]
l.sort(key=lambda x: x[0])
l

[[0, 1, 'f'], [4, 2, 't'], [9, 4, 'afsd']]

In [49]:
def genieLinkage2(data, limiar, clusterNumber):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()
    
    dists = []
    
    for i in range(0, data.shape[0]):
        for j in range(i, data.shape[1]):
            dists.append(([data[i][j], i, j]))
    
    dists.sort(key=lambda x: x[0])
    pos = 0

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(counting(clusters[k-1])) <= limiar):
            for i in range(pos, len(dists)):
                if(dists[i][0] == data[dists[i][1]][dists[i][2]]):
                    pos = i
                    min_val = dists[i][0]
                    row_index = dists[i][1]
                    col_index = dists[i][2]
                    break
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (len(set(clusters[k])) == clusterNumber):
            return clusters[k]
        

In [13]:
def genie(dataset, target, limiar, clusterNumber):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.values
    dataY = target[0]
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage(dataDistances, limiar, clusterNumber)
    
    labels = labelClusters(clusters, dataY)
    
    fmScore = f1_score(dataY, labels, average='micro')
    
    return fmScore

In [31]:
def genie2(dataset, target, limiar, clusterNumber):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.values
    dataY = target[0]
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage2(dataDistances, limiar, clusterNumber)
    
    labels = labelClusters(clusters, dataY)
    
    fmScore = f1_score(dataY, labels, average='micro')
    
    return fmScore

In [16]:
a1 = pd.read_table("datasets/a1.data", header=None, delim_whitespace=True)
iris5 = pd.read_table("datasets/iris5.data", header=None, delim_whitespace=True)
flame = pd.read_table("datasets/flame.data", header=None, delim_whitespace=True)
pathbased = pd.read_table("datasets/pathbased.data", header=None, delim_whitespace=True)
unbalance = pd.read_table("datasets/unbalance.data", header=None, delim_whitespace=True)

In [17]:
a1labels = pd.read_table("labels/a1.labels", header=None, delim_whitespace=True)
iris5labels = pd.read_table("labels/iris5.labels", header=None, delim_whitespace=True)
flamelabels = pd.read_table("labels/flame.labels", header=None, delim_whitespace=True)
pathbasedlabels = pd.read_table("labels/pathbased.labels", header=None, delim_whitespace=True)
unbalancelabels = pd.read_table("labels/unbalance.labels", header=None, delim_whitespace=True)

In [26]:
import time

In [29]:
t1 = time.time()
print(genie(iris5, iris5labels, 0.3, 3))
t2 = time.time()
print(t2 - t1)

0.895238095238
0.3586111068725586


In [51]:
t1 = time.time()
print(genie2(iris5, iris5labels, 0.3, 3))
t2 = time.time()
print(t2 - t1)

0.895238095238
0.23567605018615723


In [56]:
t1 = time.time()
print(genie(pathbased, pathbasedlabels, 0.6, 3))
t2 = time.time()
print(t2 - t1)

0.823333333333
9.737985134124756


In [57]:
t1 = time.time()
print(genie2(pathbased, pathbasedlabels, 0.6, 3))
t2 = time.time()
print(t2 - t1)

0.823333333333
4.458505868911743


In [74]:
iris5data = []
for i in range(1,7):
    if (i==1):
        iris5data.append((genie2(iris5, iris5labels, 1, 3)))
    else:
        iris5data.append((genie2(iris5, iris5labels, i/10, 3)))
iris5data

[0.54285714285714282,
 0.89523809523809528,
 0.89523809523809528,
 0.89523809523809528,
 0.5714285714285714,
 0.5714285714285714]

In [101]:
# só executar da primeira vez

# result = pd.DataFrame(columns = ["single","gini_02","gini_03","gini_04","gini_05","gini_06"])
# result.loc[0] = iris5data
# result = result.rename({0:'iris5'})
# result.to_csv("results/result.csv")

In [103]:
def computeScore(base):
    result = pd.read_csv("results/result.csv", index_col=0)
    
    data = pd.read_table("datasets/"+base+".data", header=None, delim_whitespace=True)
    labels = pd.read_table("labels/"+base+".labels", header=None, delim_whitespace=True)
    k = len(set(labels[0]))
    
    resultData = []
    for i in range(1,7):
        if (i==1):
            resultData.append((genie2(data, labels, 1, k)))
        else:
            resultData.append((genie2(data, labels, i/10, k)))
    
    size = len(result)
    result.loc[size] = resultData
    result = result.rename({size:base})
    result.to_csv("results/result.csv")

In [104]:
computeScore("iris")

In [108]:
result = pd.read_csv("results/result.csv", index_col=0)
result

,single,gini_02,gini_03,gini_04,gini_05,gini_06
iris5,0.542857,0.895238,0.895238,0.895238,0.571429,0.571429
iris,0.680000,0.960000,0.960000,0.960000,0.700000,0.693333
Aggregation,0.827411,0.790609,0.847716,0.870558,0.911168,0.827411


In [107]:
computeScore("Aggregation")